#Introduction

This is a very silly Jupyter Notebook designed to maximize your Dungeons & Dragons play and get the most out of your Necromancy Wizard.  The idea for this notebook came from one of my current 5th edition D&D campaigns with my friends where my character is a 10th level Necromancer.  Our party needed to defend a town from an oncoming seige and I needed to know just how many skeletons I could bring back from the dead in 4 days so that when the corrupt sheriff arrives to destroy the town, we would have an army of the undead to aid us in the battle.  

#The Rules 

For those uninitiated with the D&D rules, each player creates a character which they then use to tackle the scenarios that their Dungeon Master (or DM) throws at them.  Characters have specific resources allocated to them based on the class that they choose to play.  As a Wizard, you get access to a list of spells which do almost everything you can imagine: summoning meteors from the sky, changing the weather, reading text in an unknown language, and yes, bringing people back from the dead.  The main resource at a Wizard's disposal is their spell slots, which is the cost that they pay to cast a particular spell.  Spells are leveled (from 1-9) and characters are allocated a particular number of spell slots for each corresponding level, each slot allowing them to cast 1 spell of that level.   Upcasting spells is also allowed, meaning that you can use a higher level slot to cast a lower level spell, sometimes there are added benefits to casting a lower level spell with a higher level slot.  There is one spell that I was primarily concerned with ~abusing~ maximizing. This is the 'Animate Dead' spell, the rules of which I have reproduced below.  Animate Dead
3 Necromancy

    Casting Time: 1 minute
    Range: 10 feet
    Target: A pile of bones or a corpse of a Medium or Small humanoid within range
    Components: V S M (A drop of blood, a piece of flesh, and a pinch of bone dust)
    Duration: Instantaneous
    Classes: Cleric, Wizard
    This spell creates an undead servant. Choose a pile of bones or a corpse of a Medium or Small humanoid within range. Your spell imbues the target with a foul mimicry of life, raising it as an undead creature. The target becomes a skeleton if you chose bones or a zombie if you chose a corpse (the GM has the creature’s game statistics). On each of your turns, you can use a bonus action to mentally command any creature you made with this spell if the creature is within 60 feet of you (if you control multiple creatures, you can command any or all of them at the same time, issuing the same command to each one). You decide what action the creature will take and where it will move during its next turn, or you can issue a general command, such as to guard a particular chamber or corridor. If you issue no commands, the creature only defends itself against hostile creatures. Once given an order, the creature continues to follow it until its task is complete. The creature is under your control for 24 hours, after which it stops obeying any command you’ve given it. To maintain control of the creature for another 24 hours, you must cast this spell on the creature again before the current 24-hour period ends. This use of the spell reasserts your control over up to four creatures you have animated with this spell, rather than animating a new one.
    At Higher Levels: When you cast this spell using a spell slot of 4th level or higher, you animate or reassert control over two additional Undead creatures for each slot above 3rd. Each of the creatures must come from a different corpse or pile of bones.

The following code is an attempt to solve a linear optimization problem to maximize the total number of skeletons my character could reasonably create using all of the spell slot resources available to her each day.  Wizards additionally have a special ability called 'Arcane Recovery' which allows them to recover some number of spell slots (dependent on their level) once per day.  I have ignored this ability for simplicity, but hope to integrate it into the mathematics in the future.  
 

The first step is to define a matrix of weights which can be used later for the optimization problem.  The idea here is to weight the use of each spell slot based on the potential benefits that they could give for either the application of animating or reasserting control (reanimating) a skeleton.  I create a dictionary which tells me the number of skeletons I can (animate, reanimate) per spell slot and use this later for my weights.  As seen in the reanimate() function, to encompass the idea that a higher level slot is more valuable, I create my cost function such that it weighs the sum of the spell slots used against this potential benefit. 

In [1]:
from scipy import optimize as opt
import numpy as np

spell_slots = {
    3: 3,
    4: 3,
    5: 2,
    6: 1,
}; 

animate_at3 = 2
reanimate_at3 = 5
bonus = 2

n_days = 4

n_skellies_per_slot = {}
for i in range(3, 7): 
        n_skellies_per_slot[i] = (animate_at3 + (i-3)*bonus, reanimate_at3 + (i-3)*bonus)


This next cell is where most of the magic happens (It's where the wizard casts the spell).  I break out the total number of spell slots into 8 different varaibles, dividing up spell slots that are used for initial animation and reanimation, and constraining the optimization problem such that, no matter how these slots are allocated (either creating new skeletons or keeping the old ones) that I never use more than my available slots.  I add an additional constraint such that the number of skeletons that are reanimated each day does not exceed the total number of skeletons that were accumulated from the previous days.   This avoids the tremendously silly case where my wizard starts with 0 skeletons, makes no new ones, and uses all of her resources to reassert control over 0 skeletons.  

In [6]:
def reanimate(n_skellies,curr_slots): 
    """ Solves optimization problem to balance use of spell slots with resurrecting all the skellies"""
    def f(x,w1,w2): 
        return -np.matmul(w1,x[:4]) - np.matmul(w2,x[4:]) + np.sum(x)
    
    w1 = np.zeros(4)
    w2 = np.zeros(4)
    for lvl, (a,r) in n_skellies_per_slot.items():
        w1[lvl-3] = r
        w2[lvl-3] = a
    bounds = opt.Bounds(np.zeros(8), np.hstack((curr_slots, curr_slots)))
    m = [[1,0,0,0,1,0,0,0],
         [0,1,0,0,0,1,0,0],
         [0,0,1,0,0,0,1,0],
         [0,0,0,1,0,0,0,1],
         [w1[0],w1[1],w1[2],w1[3],0,0,0,0]]
    constraint = opt.LinearConstraint(m, lb=[0,0,0,0,0], ub=np.hstack((curr_slots,n_skellies)))
    x_start = np.asarray([1,1,1,1,1,1,1,1])
    result = opt.minimize(f, x_start, args=(w1,w2), method='trust-constr', bounds=bounds, constraints=constraint)
        
    

    return (np.matmul(np.hstack((w1,w2)),np.round(result.x)), np.round(result.x))

This final cell is where the optimization problem is actually run.  It keeps track of the total number of accumulated skeletns and prints out the new running total of skeletons each day as well as the spell slots utilized for those skeletons.  The array of spell slots is printed as first showing the slots (level 3-6) used for reanimating old skeletons followed by the spell slots (level 3-6) used for creating new skeletons.  

In [7]:

n_skellies = 0
for i in range(n_days): 
    print(f"Day {i+1}")
    #regain spell slots
    curr_slots = np.asarray(list(spell_slots.values()))
    n_skellies, result = reanimate(n_skellies, curr_slots)

    print(f"{n_skellies} skeletons, Spell Slots used: {result} ")
 
    
    

Day 1
38.0 skeletons, Spell Slots used: [ 0.  0. -0. -0.  3.  3.  2.  1.] 
Day 2
56.0 skeletons, Spell Slots used: [3. 3. 0. 0. 0. 0. 2. 1.] 
Day 3
62.0 skeletons, Spell Slots used: [3. 3. 2. 0. 0. 0. 0. 1.] 
Day 4
65.0 skeletons, Spell Slots used: [3. 3. 2. 1. 0. 0. 0. 0.] 


Turns out my Necromancer could have a batallion of 65 skeletons ready in 4 days.  